In [4]:
from utilsml import train_model, prepare_data
from utilsdf import list_files_by_tilenames
from uvars import tilenames_lidar,RES_DPATH
from utilsdf import assign_nulls, fillna,dropnulls_bycol,check_fillnulls, list_files_by_tilenames
from uvars import s1_fnames, s2_fnames,aux_names
import pandas as pd 
from sklearn.model_selection import train_test_split

target_col = tcol = 'edem' # run for all 3 targets, and also use zdiff or not if going to donwscale 
features_col = aux_names + s1_fnames +s2_fnames
tilenames = tilenames_lidar
X=90

In [2]:


fparquet_list,tile_files_list = list_files_by_tilenames(RES_DPATH, X, tilenames)
df = pd.read_parquet(fparquet_list[1])
df = assign_nulls(df)
df = fillna(df, s1_fnames)
df = fillna(df, s2_fnames)
df =  dropnulls_bycol(df, col=tcol) # use this only latter, fill hem always
df = check_fillnulls(df) # these two could be used later depending on the  tcol

In [3]:
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=43)
model_type_list = ["catboost","xgboost","lightgbm"]

In [6]:
X_train, y_train, X_valid, y_valid = prepare_data(train_df, valid_df, target_col, features_col)

In [10]:
X_regr_train = X_train.copy()
X_regr_valid = X_valid.copy()
y_regr_train = y_train.copy()
y_regr_valid = y_valid.copy()

In [11]:
import numpy as np
import pandas as pd
from scipy import stats

from sklearn.model_selection import train_test_split
from hyperopt import hp
from hyperopt import Trials
from shaphypetune import BoostSearch, BoostBoruta, BoostRFE, BoostRFA
from lightgbm import *
import warnings
warnings.simplefilter('ignore')

In [24]:
param_grid = {
    'learning_rate': [0.2, 0.1],
    'num_leaves': [25, 35],
    'max_depth': [10, 12]
}

param_dist = {
    'learning_rate': stats.uniform(0.09, 0.25),
    'num_leaves': stats.randint(20,40),
    'max_depth': [10, 12]
}

param_dist_hyperopt = {
    'max_depth': 15 + hp.randint('num_leaves', 5), 
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
}


In [25]:

regr_lgbm = LGBMRegressor(n_estimators=150, random_state=0, n_jobs=-1)

HYPERPARAM TUNING WITH GRID-SEARCH

In [26]:
from lightgbm import early_stopping, log_evaluation

# Define the LGBMRegressor
regr_lgbm = LGBMRegressor(n_estimators=150, random_state=0, n_jobs=-1)

# Ensure the BoostSearch class supports custom callbacks
model = BoostSearch(regr_lgbm, param_grid=param_grid)

# Fit the model with callbacks for early stopping and optional logging
model.fit(
    X_train, 
    y_train, 
    eval_set=[(X_valid, y_valid)],  # Validation set for monitoring
    eval_metric="rmse",  # Optional: specify an evaluation metric
    callbacks=[
        early_stopping(stopping_rounds=10),  # Stop if no improvement after 10 rounds
        log_evaluation(period=1)  # Log evaluation metrics every iteration
    ]
)



8 trials detected for ('learning_rate', 'num_leaves', 'max_depth')

trial: 0001 ### iterations: 00150 ### eval_score: 0.24835
trial: 0002 ### iterations: 00150 ### eval_score: 0.24893
trial: 0003 ### iterations: 00150 ### eval_score: 0.23919
trial: 0004 ### iterations: 00150 ### eval_score: 0.23758
trial: 0005 ### iterations: 00150 ### eval_score: 0.26949
trial: 0006 ### iterations: 00150 ### eval_score: 0.26856
trial: 0007 ### iterations: 00150 ### eval_score: 0.25576
trial: 0008 ### iterations: 00150 ### eval_score: 0.25481


BoostSearch(estimator=LGBMRegressor(n_estimators=150, n_jobs=-1,
                                    random_state=0),
            param_grid={'learning_rate': [0.2, 0.1], 'max_depth': [10, 12],
                        'num_leaves': [25, 35]})

In [27]:
model.estimator_, model.best_params_, model.best_score_,model.score(X_valid, y_valid)

(LGBMRegressor(learning_rate=0.2, max_depth=12, n_estimators=150, n_jobs=-1,
               num_leaves=35, random_state=0),
 {'learning_rate': 0.2, 'num_leaves': 35, 'max_depth': 12},
 0.2375772792221409)

In [28]:
model.score(X_valid, y_valid)

0.891410240950568

HYPERPARAM TUNING WITH RANDOM-SEARCH

In [31]:
from lightgbm import early_stopping, log_evaluation

# Define the LGBMRegressor
regr_lgbm = LGBMRegressor(n_estimators=150, random_state=0, n_jobs=-1)

# Ensure the BoostSearch class supports custom callbacks
model = BoostSearch(regr_lgbm, param_grid=param_dist,
                    n_iter=8, sampling_seed=0)

# Fit the model with callbacks for early stopping and optional logging
model.fit(
    X_train, 
    y_train, 
    eval_set=[(X_valid, y_valid)],  # Validation set for monitoring
    eval_metric="rmse",  # Optional: specify an evaluation metric
    callbacks=[
        early_stopping(stopping_rounds=10),  # Stop if no improvement after 10 rounds
        log_evaluation(period=1)  # Log evaluation metrics every iteration
    ]
)



8 trials detected for ('learning_rate', 'num_leaves', 'max_depth')

trial: 0001 ### iterations: 00150 ### eval_score: 0.24745
trial: 0002 ### iterations: 00150 ### eval_score: 0.24581
trial: 0003 ### iterations: 00150 ### eval_score: 0.25623
trial: 0004 ### iterations: 00150 ### eval_score: 0.25396
trial: 0005 ### iterations: 00150 ### eval_score: 0.25202
trial: 0006 ### iterations: 00150 ### eval_score: 0.23085
trial: 0007 ### iterations: 00150 ### eval_score: 0.25895
trial: 0008 ### iterations: 00150 ### eval_score: 0.23801


BoostSearch(estimator=LGBMRegressor(n_estimators=150, n_jobs=-1,
                                    random_state=0),
            n_iter=8,
            param_grid={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x74868faaff10>,
                        'max_depth': [10, 12],
                        'num_leaves': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7486980a4f90>},
            sampling_seed=0)

In [32]:
model.estimator_, model.best_params_, model.best_score_,model.score(X_valid, y_valid)

(LGBMRegressor(learning_rate=0.30166560180253343, max_depth=12, n_estimators=150,
               n_jobs=-1, num_leaves=37, random_state=0),
 {'learning_rate': 0.30166560180253343, 'num_leaves': 37, 'max_depth': 12},
 0.2308496465924432,
 0.8944852488327354)

HYPERPARAM TUNING WITH HYPEROPT

In [34]:
from hyperopt import Trials
from lightgbm import early_stopping, log_evaluation

# Define the LGBMRegressor
regr_lgbm = LGBMRegressor(n_estimators=150, random_state=0, n_jobs=-1)

# Define the BoostSearch with Hyperopt parameters
model = BoostSearch(
    regr_lgbm, 
    param_grid=param_dist_hyperopt, 
    n_iter=8, 
    sampling_seed=0
)

# Create a Trials object for Hyperopt
trials = Trials()

# Fit the model with Hyperopt, using callbacks for early stopping and logging
model.fit(
    X_train, 
    y_train, 
    trials=trials,  # Pass the Trials object here
    eval_set=[(X_valid, y_valid)],  # Validation set
    eval_metric="rmse",  # Optional: specify evaluation metric
    callbacks=[
        early_stopping(stopping_rounds=10),  # Stop if no improvement after 10 rounds
        log_evaluation(period=1)  # Log evaluation metrics every iteration
    ]
)



8 trials detected for ('max_depth', 'learning_rate', 'colsample_bytree')

trial: 0001 ### iterations: 00150 ### eval_score: 0.29902
trial: 0002 ### iterations: 00150 ### eval_score: 0.24948
trial: 0003 ### iterations: 00150 ### eval_score: 0.24473
trial: 0004 ### iterations: 00150 ### eval_score: 0.29778
trial: 0005 ### iterations: 00150 ### eval_score: 0.34209
trial: 0006 ### iterations: 00150 ### eval_score: 0.27614
trial: 0007 ### iterations: 00150 ### eval_score: 0.24069
trial: 0008 ### iterations: 00150 ### eval_score: 0.26919


BoostSearch(estimator=LGBMRegressor(n_estimators=150, n_jobs=-1,
                                    random_state=0),
            n_iter=8,
            param_grid={'colsample_bytree': <hyperopt.pyll.base.Apply object at 0x7486980a3150>,
                        'learning_rate': <hyperopt.pyll.base.Apply object at 0x74868feabe50>,
                        'max_depth': <hyperopt.pyll.base.Apply object at 0x74868fb6fd90>},
            sampling_seed=0)

In [35]:
model.estimator_, model.best_params_, model.best_score_,model.score(X_valid, y_valid)

(LGBMRegressor(colsample_bytree=0.8933626813491535,
               learning_rate=0.19481043968099895, max_depth=17, n_estimators=150,
               n_jobs=-1, random_state=0),
 {'colsample_bytree': 0.8933626813491535,
  'learning_rate': 0.19481043968099895,
  'max_depth': 17},
 0.24068640891192444,
 0.8899891469597098)

## Features Selection

BORUTA

In [36]:
model = BoostBoruta(regr_lgbm, max_iter=200, perc=100)
model.fit(
    X_train, 
    y_train, 
    eval_set=[(X_valid, y_valid)],  # Validation set
    eval_metric="rmse",  # Optional: specify evaluation metric
    callbacks=[
        early_stopping(stopping_rounds=10),  # Stop if no improvement after 10 rounds
        log_evaluation(period=1)  # Log evaluation metrics every iteration
    ]
)


BoostBoruta(estimator=LGBMRegressor(n_estimators=150, n_jobs=-1,
                                    random_state=0),
            max_iter=200)

In [40]:
def print_params(model):
    print('')
    print('estimator')
    print(model.estimator_)
    print('features num', model.n_features_)
    print('feature ranking')
    print(model.ranking_)
    print('Score',model.score(X_valid, y_valid))

In [41]:
print_params(model)


estimator
LGBMRegressor(n_estimators=150, n_jobs=-1, random_state=0)
features num 16
feature ranking
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Score 0.8809152669270383


RECURSIVE FEATURE ELIMINATION (RFE)

In [42]:
model = BoostRFE(regr_lgbm, min_features_to_select=1,step=1)
model.fit(
    X_train, 
    y_train, 
    eval_set=[(X_valid, y_valid)],  # Validation set
    eval_metric="rmse",  # Optional: specify evaluation metric
    callbacks=[
        early_stopping(stopping_rounds=10),  # Stop if no improvement after 10 rounds
        log_evaluation(period=1)  # Log evaluation metrics every iteration
    ]
)
print_params(model)


estimator
LGBMRegressor(n_estimators=150, n_jobs=-1, random_state=0)
features num 9
feature ranking
[1 1 1 1 1 5 1 2 1 7 8 6 4 3 1 1]
Score 0.8817759786698391


In [43]:
model = BoostRFA(regr_lgbm, min_features_to_select=1, step=1)
model.fit(
    X_train, 
    y_train, 
    eval_set=[(X_valid, y_valid)],  # Validation set
    eval_metric="rmse",  # Optional: specify evaluation metric
    callbacks=[
        early_stopping(stopping_rounds=10),  # Stop if no improvement after 10 rounds
        log_evaluation(period=1)  # Log evaluation metrics every iteration
    ]
)

BoostRFA(estimator=LGBMRegressor(n_estimators=150, n_jobs=-1, random_state=0),
         min_features_to_select=1)

In [49]:
def print_params(model, X_valid, y_valid):
    """
    Print details of the model's parameters, feature importance, and performance.

    Parameters:
    - model: The fitted model (must have attributes like estimator_, n_features_, ranking_, and score).
    - X_valid: Validation feature set (must be a DataFrame).
    - y_valid: Validation target set.
    """
    print("\nModel Parameters and Performance:")
    print("-" * 40)
    
    # Check and print the estimator
    if hasattr(model, "estimator_"):
        print("Estimator:")
        print(model.estimator_)
    else:
        print("Estimator: Not available")
    
    # Check and print the number of features
    if hasattr(model, "n_features_"):
        print(f"Number of Features: {model.n_features_}")
    else:
        print("Number of Features: Not available")
    
    # Check and print feature ranking
    if hasattr(model, "ranking_"):
        try:
            feature_ranking = dict(zip(X_valid.columns.tolist(), model.ranking_))
            print("Feature Ranking (Feature: Rank):")
            for feature, rank in sorted(feature_ranking.items(), key=lambda x: x[1]):
                print(f"{feature}: {rank}")
        except Exception as e:
            print(f"Feature Ranking: Could not map due to error - {e}")
    else:
        print("Feature Ranking: Not available")
    
    # Check and print the validation score
    if hasattr(model, "score"):
        try:
            score = model.score(X_valid, y_valid)
            print(f"Validation Score: {score:.4f}")
        except Exception as e:
            print(f"Validation Score: Could not calculate due to error - {e}")
    else:
        print("Validation Score: Not available")


In [50]:
print_params(model, X_valid, y_valid)# X_valid.columns.tolist())


Model Parameters and Performance:
----------------------------------------
Estimator:
LGBMRegressor(n_estimators=150, n_jobs=-1, random_state=0)
Number of Features: 15
Feature Ranking (Feature: Rank):
egm08: 1
egm96: 2
hem: 3
vh: 4
vv: 5
blue: 6
green: 7
red: 8
nir: 9
swir1: 10
endvi: 11
ndwi: 12
andwi: 13
ui: 14
ndvi: 15
nbai: 16
Validation Score: 0.8817


BORUTA SHAP

In [51]:
model = BoostBoruta(
    regr_lgbm, max_iter=200, perc=100,
    importance_type='shap_importances', train_importance=False
)

model.fit(
    X_train, 
    y_train, 
    eval_set=[(X_valid, y_valid)],  # Validation set
    eval_metric="rmse",  # Optional: specify evaluation metric
    callbacks=[
        early_stopping(stopping_rounds=10),  # Stop if no improvement after 10 rounds
        log_evaluation(period=1)  # Log evaluation metrics every iteration
    ]
)

print_params(model, X_valid, y_valid)


Model Parameters and Performance:
----------------------------------------
Estimator:
LGBMRegressor(n_estimators=150, n_jobs=-1, random_state=0)
Number of Features: 16
Feature Ranking (Feature: Rank):
egm08: 1
egm96: 1
hem: 1
vv: 1
vh: 1
red: 1
green: 1
blue: 1
nir: 1
swir1: 1
ndvi: 1
endvi: 1
ndwi: 1
andwi: 1
nbai: 1
ui: 1
Validation Score: 0.8809


RECURSIVE FEATURE ELIMINATION (RFE) SHAP

In [52]:
def print_params(model, X_valid, y_valid):
    """
    Display detailed information about the model's parameters, feature importance, and performance.

    Parameters:
    - model: The fitted model (must have attributes like `estimator_`, `n_features_`, `ranking_`, and `score`).
    - X_valid: Validation feature set (must be a DataFrame).
    - y_valid: Validation target set.
    """
    print("\n========== Model Parameters and Performance ==========\n")
    
    # Display the model's estimator
    if hasattr(model, "estimator_"):
        print("** Model Estimator **")
        print(model.estimator_, "\n")
    else:
        print("** Model Estimator **")
        print("Information not available.\n")
    
    # Display the number of features
    if hasattr(model, "n_features_"):
        print(f"** Number of Features **\n{model.n_features_}\n")
    else:
        print("** Number of Features **\nInformation not available.\n")
    
    # Display feature rankings
    if hasattr(model, "ranking_"):
        try:
            feature_ranking = dict(zip(X_valid.columns.tolist(), model.ranking_))
            print("** Feature Ranking (Sorted by Importance) **")
            print("{:<30}{}".format("Feature Name", "Rank"))
            print("-" * 40)
            for feature, rank in sorted(feature_ranking.items(), key=lambda x: x[1]):
                print(f"{feature:<30}{rank}")
            print("")
        except Exception as e:
            print(f"** Feature Ranking **\nUnable to map feature names to rankings due to error: {e}\n")
    else:
        print("** Feature Ranking **\nInformation not available.\n")
    
    # Display validation score
    if hasattr(model, "score"):
        try:
            score = model.score(X_valid, y_valid)
            print(f"** Validation Score **\n{score:.4f}\n")
        except Exception as e:
            print(f"** Validation Score **\nCould not compute due to error: {e}\n")
    else:
        print("** Validation Score **\nInformation not available.\n")
    
    print("=" * 50)

print_params(model, X_valid, y_valid)



========== Model Parameters and Performance ==========

** Model Estimator **
LGBMRegressor(n_estimators=150, n_jobs=-1, random_state=0) 

** Number of Features **
16

** Feature Ranking (Sorted by Importance) **
Feature Name                  Rank
----------------------------------------
egm08                         1
egm96                         1
hem                           1
vv                            1
vh                            1
red                           1
green                         1
blue                          1
nir                           1
swir1                         1
ndvi                          1
endvi                         1
ndwi                          1
andwi                         1
nbai                          1
ui                            1

** Validation Score **
0.8809



In [53]:
model = BoostBoruta(
    regr_lgbm, max_iter=200, perc=100,
    importance_type='shap_importances', train_importance=False
)

model.fit(
    X_train, 
    y_train, 
    eval_set=[(X_valid, y_valid)],  # Validation set
    eval_metric="rmse",  # Optional: specify evaluation metric
    callbacks=[
        early_stopping(stopping_rounds=10),  # Stop if no improvement after 10 rounds
        log_evaluation(period=1)  # Log evaluation metrics every iteration
    ]
)

print_params(model, X_valid, y_valid)


========== Model Parameters and Performance ==========

** Model Estimator **
LGBMRegressor(n_estimators=150, n_jobs=-1, random_state=0) 

** Number of Features **
16

** Feature Ranking (Sorted by Importance) **
Feature Name                  Rank
----------------------------------------
egm08                         1
egm96                         1
hem                           1
vv                            1
vh                            1
red                           1
green                         1
blue                          1
nir                           1
swir1                         1
ndvi                          1
endvi                         1
ndwi                          1
andwi                         1
nbai                          1
ui                            1

** Validation Score **
0.8809



RECURSIVE FEATURE ADDITION (RFA) SHAP

In [54]:
model = BoostRFA(
    regr_lgbm, min_features_to_select=1, step=1,
    importance_type='shap_importances', train_importance=False
)

model.fit(
    X_train, 
    y_train, 
    eval_set=[(X_valid, y_valid)],  # Validation set
    eval_metric="rmse",  # Optional: specify evaluation metric
    callbacks=[
        early_stopping(stopping_rounds=10),  # Stop if no improvement after 10 rounds
        log_evaluation(period=1)  # Log evaluation metrics every iteration
    ]
)

print_params(model, X_valid, y_valid)


========== Model Parameters and Performance ==========

** Model Estimator **
LGBMRegressor(n_estimators=150, n_jobs=-1, random_state=0) 

** Number of Features **
14

** Feature Ranking (Sorted by Importance) **
Feature Name                  Rank
----------------------------------------
vh                            1
vv                            2
nir                           3
ndwi                          4
ndvi                          5
swir1                         6
andwi                         7
nbai                          8
endvi                         9
blue                          10
hem                           11
green                         12
egm08                         13
egm96                         14
red                           15
ui                            15

** Validation Score **
0.8813



## Hyperparameters Tuning + Features Selection

HYPERPARAM TUNING WITH GRID-SEARCH + BORUTA

In [55]:
model = BoostBoruta(regr_lgbm, param_grid=param_grid, max_iter=200, perc=100)

model.fit(
    X_train, 
    y_train, 
    eval_set=[(X_valid, y_valid)],  # Validation set
    eval_metric="rmse",  # Optional: specify evaluation metric
    callbacks=[
        early_stopping(stopping_rounds=10),  # Stop if no improvement after 10 rounds
        log_evaluation(period=1)  # Log evaluation metrics every iteration
    ]
)

print_params(model, X_valid, y_valid)


8 trials detected for ('learning_rate', 'num_leaves', 'max_depth')

trial: 0001 ### iterations: 00150 ### eval_score: 0.24835
trial: 0002 ### iterations: 00150 ### eval_score: 0.24893
trial: 0003 ### iterations: 00150 ### eval_score: 0.23919
trial: 0004 ### iterations: 00150 ### eval_score: 0.23758
trial: 0005 ### iterations: 00150 ### eval_score: 0.26949
trial: 0006 ### iterations: 00150 ### eval_score: 0.26856
trial: 0007 ### iterations: 00150 ### eval_score: 0.25576
trial: 0008 ### iterations: 00150 ### eval_score: 0.25481

========== Model Parameters and Performance ==========

** Model Estimator **
LGBMRegressor(learning_rate=0.2, max_depth=12, n_estimators=150, n_jobs=-1,
              num_leaves=35, random_state=0) 

** Number of Features **
16

** Feature Ranking (Sorted by Importance) **
Feature Name                  Rank
----------------------------------------
egm08                         1
egm96                         1
hem                           1
vv                 

HYPERPARAM TUNING WITH RANDOM-SEARCH + RECURSIVE FEATURE ELIMINATION (RFE)

In [57]:
model = BoostRFE(regr_lgbm, param_grid=param_dist, min_features_to_select=1, step=1,
                 n_iter=8, sampling_seed=0)

model.fit(
    X_train, 
    y_train, 
    eval_set=[(X_valid, y_valid)],  # Validation set
    eval_metric="rmse",  # Optional: specify evaluation metric
    callbacks=[
        early_stopping(stopping_rounds=10),  # Stop if no improvement after 10 rounds
        log_evaluation(period=1)  # Log evaluation metrics every iteration
    ]
)

print_params(model, X_valid, y_valid)


8 trials detected for ('learning_rate', 'num_leaves', 'max_depth')

trial: 0001 ### iterations: 00150 ### eval_score: 0.24577
trial: 0002 ### iterations: 00150 ### eval_score: 0.24402
trial: 0003 ### iterations: 00150 ### eval_score: 0.25503
trial: 0004 ### iterations: 00150 ### eval_score: 0.25396
trial: 0005 ### iterations: 00150 ### eval_score: 0.25059
trial: 0006 ### iterations: 00150 ### eval_score: 0.23004
trial: 0007 ### iterations: 00150 ### eval_score: 0.25681
trial: 0008 ### iterations: 00150 ### eval_score: 0.23734

========== Model Parameters and Performance ==========

** Model Estimator **
LGBMRegressor(learning_rate=0.30166560180253343, max_depth=12, n_estimators=150,
              n_jobs=-1, num_leaves=37, random_state=0) 

** Number of Features **
13

** Feature Ranking (Sorted by Importance) **
Feature Name                  Rank
----------------------------------------
egm08                         1
egm96                         1
hem                           1
vv 

HYPERPARAM TUNING WITH HYPEROPT + RECURSIVE FEATURE ADDITION (RFA)

In [58]:
model = BoostRFA(regr_lgbm, param_grid=param_dist_hyperopt, min_features_to_select=1, step=1,
                 n_iter=8, sampling_seed=0)

trials = Trials()
model.fit(
    X_train, 
    y_train, 
    trials=trials,  # Pass the Trials object here
    eval_set=[(X_valid, y_valid)],  # Validation set
    eval_metric="rmse",  # Optional: specify evaluation metric
    callbacks=[
        early_stopping(stopping_rounds=10),  # Stop if no improvement after 10 rounds
        log_evaluation(period=1)  # Log evaluation metrics every iteration
    ]
)

print_params(model, X_valid, y_valid)


8 trials detected for ('max_depth', 'learning_rate', 'colsample_bytree')

trial: 0001 ### iterations: 00150 ### eval_score: 0.30086
trial: 0002 ### iterations: 00150 ### eval_score: 0.24979
trial: 0003 ### iterations: 00150 ### eval_score: 0.24395
trial: 0004 ### iterations: 00150 ### eval_score: 0.29561
trial: 0005 ### iterations: 00150 ### eval_score: 0.3398
trial: 0006 ### iterations: 00150 ### eval_score: 0.27544
trial: 0007 ### iterations: 00150 ### eval_score: 0.24268
trial: 0008 ### iterations: 00150 ### eval_score: 0.26789

========== Model Parameters and Performance ==========

** Model Estimator **
LGBMRegressor(colsample_bytree=0.8933626813491535,
              learning_rate=0.19481043968099895, max_depth=17, n_estimators=150,
              n_jobs=-1, random_state=0) 

** Number of Features **
12

** Feature Ranking (Sorted by Importance) **
Feature Name                  Rank
----------------------------------------
egm08                         1
egm96                     

HYPERPARAM TUNING WITH GRID-SEARCH + BORUTA SHAP

In [56]:
#https://www.kaggle.com/code/towhidultonmoy/shap-hypertune-everything-you-need-to-know

In [ ]:
# comavere the 3 methods outcome in table with RMSE and R2 in predictions 